In [129]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

downloads = "C:/Users/stucws/Downloads/"
add = 'macro_indicators/'

In [ ]:
# disc = pd.read_csv(downloads + 'TWEXB.csv')

broad = pd.read_csv(downloads + add + 'US_Dollar_Index_Broad.csv')
broad['DTWEXBGS'] = pd.to_numeric(broad['DTWEXBGS'], errors='coerce')
broad['DTWEXBGS'] = broad['DTWEXBGS'].fillna(broad['DTWEXBGS'].rolling(window=3, center=True, min_periods=1).mean())
broad['DTWEXBGS'] = broad['DTWEXBGS'].fillna(broad['DTWEXBGS'].rolling(window=4, center=True, min_periods=1).mean())

adv = pd.read_csv(downloads + add + 'US_Dollar_Index_ADV.csv')
adv['DTWEXAFEGS'] = pd.to_numeric(adv['DTWEXAFEGS'], errors='coerce')
adv['DTWEXAFEGS'] = adv['DTWEXAFEGS'].fillna(adv['DTWEXAFEGS'].rolling(window=3, center=True, min_periods=1).mean())
adv['DTWEXAFEGS'] = adv['DTWEXAFEGS'].fillna(adv['DTWEXAFEGS'].rolling(window=4, center=True, min_periods=1).mean())

emg = pd.read_csv(downloads + add + 'US_Dollar_Index_EMG.csv')
emg['DTWEXEMEGS'] = pd.to_numeric(emg['DTWEXEMEGS'], errors='coerce')
emg['DTWEXEMEGS'] = emg['DTWEXEMEGS'].fillna(emg['DTWEXEMEGS'].rolling(window=3, center=True, min_periods=1).mean())
emg['DTWEXEMEGS'] = emg['DTWEXEMEGS'].fillna(emg['DTWEXEMEGS'].rolling(window=4, center=True, min_periods=1).mean())

merged_df = pd.merge(broad, adv, on='DATE', how='outer')
merged_df = pd.merge(merged_df, emg, on='DATE', how='outer')
merged_df.rename(columns={
    'DATE': 'Period',
    'DTWEXBGS': 'Nominal Broad Daily',
    'DTWEXAFEGS': 'Nominal AFE Daily',
    'DTWEXEMEGS': 'Nominal EME Daily'
}, inplace=True)

In [ ]:
hist = pd.read_csv(downloads + add + 'US_Dollar_Indexes_Nominal_Daily.csv')
hist['Period'] = pd.to_datetime(hist['Period'], format='%d-%b-%y')
hist['Period'] = hist['Period'].dt.strftime('%Y-%m-%d')

# merged_df = pd.merge(merged_df, hist, on='Period', how='inner')
# print(merged_df)
#
# merged_df['Nominal Broad Daily diff'] = merged_df['Nominal Broad Daily_x'] - merged_df['Nominal Broad Daily_y']
# merged_df['Nominal AFE Daily diff'] = merged_df['Nominal AFE Daily_x'] - merged_df['Nominal AFE Daily_y']
# merged_df['Nominal EME Daily diff'] = merged_df['Nominal EME Daily_x'] - merged_df['Nominal EME Daily_y']
#
# print(merged_df['Nominal Broad Daily diff'].min())
# print(merged_df['Nominal Broad Daily diff'].max())
# print(merged_df['Nominal Broad Daily diff'].mean())
# print()
#
# print(merged_df['Nominal AFE Daily diff'].min())
# print(merged_df['Nominal AFE Daily diff'].max())
# print(merged_df['Nominal AFE Daily diff'].mean())
# print()
#
# print(merged_df['Nominal EME Daily diff'].min())
# print(merged_df['Nominal EME Daily diff'].max())
# print(merged_df['Nominal EME Daily diff'].mean())

hist = hist[hist['Period'] <= '2006-01-01']
merged_df = merged_df[merged_df['Period'] > '2006-01-01']
result = pd.concat([hist, merged_df]).reset_index()

print(result)
result.to_csv(downloads + add + 'US_Dollar_Indexes_Nominal_Daily_Full.csv', index=False)

In [109]:
files = os.listdir(downloads + add)
files.remove('unnecessary')

In [125]:
all_dfs = []

for file in files:
    if file == 'US_Dollar_Indexes_Nominal_Daily_Full.csv':
        df = pd.read_csv(downloads + add + file, parse_dates=['Period'], index_col='Period')
    else:
        df = pd.read_csv(downloads + add + file, parse_dates=['DATE'], index_col='DATE')
    all_dfs.append(df)

combined_df = pd.concat(all_dfs, axis=1)

# Remove leading and trailing NAs
combined_df = combined_df.loc[lambda df_: (df_.ffill().notna() & df_.bfill().notna()).all(axis=1)]

# Remove weekends
combined_df = combined_df[combined_df.index.weekday < 5]

# Change object columns to float
combined_df['DCOILBRENTEU'] = pd.to_numeric(combined_df['DCOILBRENTEU'], errors='coerce')
combined_df['DCOILBRENTEU'] = combined_df['DCOILBRENTEU'].fillna(combined_df['DCOILBRENTEU'].rolling(window=3, center=True, min_periods=1).mean())
combined_df['DCOILBRENTEU'] = combined_df['DCOILBRENTEU'].fillna(combined_df['DCOILBRENTEU'].rolling(window=3, center=True, min_periods=1).mean())
# combined_df['DCOILBRENTEU'][combined_df['DCOILBRENTEU'].isna()]

combined_df['DCOILWTICO'] = pd.to_numeric(combined_df['DCOILWTICO'], errors='coerce')
combined_df['DCOILWTICO'] = combined_df['DCOILWTICO'].fillna(combined_df['DCOILWTICO'].rolling(window=3, center=True, min_periods=1).mean())
# combined_df['DCOILWTICO'][combined_df['DCOILWTICO'].isna()]

# forward fill the monthly values
combined_df = combined_df.ffill()
combined_df.dropna(inplace=True)

# Rename columns for visibility
combined_df.rename(columns={
    'CPIAUCSL': 'Consumer Price Index',
    'DCOILBRENTEU': 'Crude Oil (BrentEU)',
    'DCOILWTICO': 'Crude Oil (WTI)',
    'A191RL1Q225SBEA': 'GDP Real',
    'A939RX0Q048SBEA': 'GDP Real per capita',
    'GNPC96': 'GNP Real',
    'MSPUS': 'House Sales Price Median',
    'INDPRO': 'Industrial Production',
    'FPCPITOTLZGUSA': 'Inflation Rate',
    'DFF': 'Interest Rate (US)',
    'M1SL': 'M1',
    'M1V': 'M1 Velocity',
    'M2SL': 'M2',
    'M2V': 'M2 Velocity',
    'PCE': 'Personal Consumption Expenditures',
    'POPTHM': 'Population (US)',
    'SPPOPTOTLWLD': 'Population (World)',
    'TENEXPCHAREARISPRE': 'Risk Premium Real',
    'BOGZ1FA895050005Q': 'Total Capital Expenditures',
    'UNRATE': 'Unemployment Rate'
}, inplace=True)

In [128]:
combined_df.to_csv(downloads + add + 'combined_macro_indicators.csv', index_label='Date')

In [33]:
def find_column_with_smallest_nonnull_range(df_):
    max_date = datetime.datetime.max
    min_col = None
    
    for col in df_.columns:
        non_null_indices = df_[col].dropna().index
        if non_null_indices.empty:
            continue
            
        last = non_null_indices[-1]
        
        if last < max_date:
            max_date = last
            min_col = col
    
    return min_col, max_date

find_column_with_smallest_nonnull_range(combined_df)

('FPCPITOTLZGUSA', Timestamp('2023-01-01 00:00:00'))